In [1]:

import sys
import os
from pathlib import Path

# Get absolute path to project root
project_root = Path(os.getcwd()).resolve()
if project_root.name != "ncg87-blockchain_tracker":
    project_root = project_root.parent  # Adjust if running from a subdirectory

sys.path.append(str(project_root))

# Add to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from config import Settings
from chains import EthereumQuerier, EthereumParser
from database import Database


In [2]:
database = Database()
querier = EthereumQuerier()
parser = EthereumParser(database, querier)


In [3]:
block = querier.get_block()

In [4]:
parser.process_block(block)


Error inserting transaction 14fd56dbdb948efbdd3d7216cc79c1ac0115cc7aed54b798eb43f133683774f4: Python int too large to convert to SQLite INTEGER


In [12]:
import sqlite3

# Connect to your database
connection = sqlite3.connect('blockchain_data.db')
cursor = connection.cursor()

# Query
query = """
    SELECT 
        COUNT(*) AS total_entries,
        SUM(CASE WHEN input_data LIKE '{"function":%' THEN 1 ELSE 0 END) AS decoded_entries,
        SUM(CASE WHEN input_data NOT LIKE '{"function":%' AND input_data IS NOT NULL THEN 1 ELSE 0 END) AS raw_entries,
        SUM(CASE WHEN input_data IS NULL THEN 1 ELSE 0 END) AS empty_entries
    FROM transactions;
"""

# Execute query
cursor.execute(query)
result = cursor.fetchone()

# Process results
total_entries, decoded_entries, raw_entries, empty_entries = result
print(f"Total Entries: {total_entries}")
print(f"Decoded Entries: {decoded_entries}")
print(f"Raw Input Entries: {raw_entries}")
print(f"Empty Entries: {empty_entries}")

# Close connection
connection.close()


Total Entries: 275
Decoded Entries: 68
Raw Input Entries: 126
Empty Entries: 81


In [14]:
import sqlite3

# Connect to your database
connection = sqlite3.connect('blockchain_data.db')
cursor = connection.cursor()

# Query to fetch all decoded entries
query = """
    SELECT *
    FROM transactions
    WHERE input_data LIKE '{"function":%';
"""

# Execute the query
cursor.execute(query)

# Fetch all results
decoded_entries = cursor.fetchall()

# Optionally, format the results into a list of dictionaries
columns = [col[0] for col in cursor.description]
decoded_list = [dict(zip(columns, row)) for row in decoded_entries]

# Print or process the list
print(decoded_list)

# Close the connection
connection.close()


[{'id': 8, 'network': 'Ethereum', 'block_number': 21568008, 'transaction_hash': '798e50012d79a849131b51976eac8a0ce875c49558174b44421a7b4702b6a6a3', 'from_address': '0x0e8D288C5612ACfa6567d9fC07fda125a8B88807', 'to_address': '0x6ef69Ba2d051761aFD38F218F0a3cF517D64A760', 'amount': 0.0, 'gas': 56548, 'gas_price': 22067065848, 'input_data': '{"function": "<Function approve(address,uint256)>", "parameters": {"spender": "0x80a64c6D7f12C47B7c66c5B4E20E72bc1FCd5d9e", "value": 115792089237316195423570985008687907853269984665640564039457584007913129639935}}', 'timestamp': 1736197715}, {'id': 10, 'network': 'Ethereum', 'block_number': 21568008, 'transaction_hash': 'c7e0be8ac6359af9ec363c41f0bcff2cb9f8eb7749b71ba02afe251ec4fca10c', 'from_address': '0xAe0332f7ea04a07fCEDdE87EF3622b6079C76A9E', 'to_address': '0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D', 'amount': 0.0, 'gas': 600000, 'gas_price': 34067065848, 'input_data': '{"function": "<Function swapExactTokensForETHSupportingFeeOnTransferTokens(ui

In [15]:
decoded_list

[{'id': 8,
  'network': 'Ethereum',
  'block_number': 21568008,
  'transaction_hash': '798e50012d79a849131b51976eac8a0ce875c49558174b44421a7b4702b6a6a3',
  'from_address': '0x0e8D288C5612ACfa6567d9fC07fda125a8B88807',
  'to_address': '0x6ef69Ba2d051761aFD38F218F0a3cF517D64A760',
  'amount': 0.0,
  'gas': 56548,
  'gas_price': 22067065848,
  'input_data': '{"function": "<Function approve(address,uint256)>", "parameters": {"spender": "0x80a64c6D7f12C47B7c66c5B4E20E72bc1FCd5d9e", "value": 115792089237316195423570985008687907853269984665640564039457584007913129639935}}',
  'timestamp': 1736197715},
 {'id': 10,
  'network': 'Ethereum',
  'block_number': 21568008,
  'transaction_hash': 'c7e0be8ac6359af9ec363c41f0bcff2cb9f8eb7749b71ba02afe251ec4fca10c',
  'from_address': '0xAe0332f7ea04a07fCEDdE87EF3622b6079C76A9E',
  'to_address': '0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D',
  'amount': 0.0,
  'gas': 600000,
  'gas_price': 34067065848,
  'input_data': '{"function": "<Function swapExactToken

In [17]:
# Connect to your database
connection = sqlite3.connect('blockchain_data.db')
cursor = connection.cursor()

# Query to get entries with amount > 0
query = """
    SELECT *
    FROM transactions
    WHERE amount > 0;
"""

# Execute the query
cursor.execute(query)

# Fetch all results
entries_with_value = cursor.fetchall()

# Optionally, format the results into a list of dictionaries
columns = [col[0] for col in cursor.description]
formatted_entries = [dict(zip(columns, row)) for row in entries_with_value]

# Print or process the results
print(formatted_entries)

# Close the connection
connection.close()


[{'id': 1, 'network': 'Ethereum', 'block_number': 21568008, 'transaction_hash': 'ce8208003adb2f1af72e27cae96ddae417cb2cfaae57b99e9eea3ad5e9c4c813', 'from_address': '0x36a454aEF52938c8637CD4689B2980C1Cfd43389', 'to_address': '0xA69babEF1cA67A37Ffaf7a485DfFF3382056e78C', 'amount': 15946496.0, 'gas': 220708, 'gas_price': 19067065848, 'input_data': '78e111f6000000000000000000000000ad759ce7f00fb4e19224327f789d5614c2ef68b1000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000c42f1c6b50000000000000000000000000000000000000000000000a302877b9596bce8d8800000000000000000000000000000000000000000000434d9907859624000000000000000000000000000000000000000000000004921e4ac8c70927176806c200000000000000000000000000000000000000000000000000012047c33af86200000000000000000000000000000000000000000000000000000000677c4653ff8000000000000000000000000000000000000000000000000000000000fd8300000000000000000000000000000000000000000000000000000000', '

In [18]:
formatted_entries

[{'id': 1,
  'network': 'Ethereum',
  'block_number': 21568008,
  'transaction_hash': 'ce8208003adb2f1af72e27cae96ddae417cb2cfaae57b99e9eea3ad5e9c4c813',
  'from_address': '0x36a454aEF52938c8637CD4689B2980C1Cfd43389',
  'to_address': '0xA69babEF1cA67A37Ffaf7a485DfFF3382056e78C',
  'amount': 15946496.0,
  'gas': 220708,
  'gas_price': 19067065848,
  'input_data': '78e111f6000000000000000000000000ad759ce7f00fb4e19224327f789d5614c2ef68b1000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000c42f1c6b50000000000000000000000000000000000000000000000a302877b9596bce8d8800000000000000000000000000000000000000000000434d9907859624000000000000000000000000000000000000000000000004921e4ac8c70927176806c200000000000000000000000000000000000000000000000000012047c33af86200000000000000000000000000000000000000000000000000000000677c4653ff8000000000000000000000000000000000000000000000000000000000fd83000000000000000000000000000000000000000000